In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-prompt-recovery-data-gemini-and-gemma/gemma_data_set_prompt_recover_1.csv
/kaggle/input/llm-prompt-recovery-data-gemini-and-gemma/gemma_data_set_prompt_recover_2.csv
/kaggle/input/llm-prompt-recovery-data-gemini-and-gemma/gemini_data_set_prompt_recover_3.csv


> Word Embedding Layers

In [2]:
# load text 

filename = '/kaggle/input/llm-prompt-recovery-data-gemini-and-gemma/gemini_data_set_prompt_recover_3.csv'
file = open(filename, 'rt')
text = file.read()
file.close()

In [8]:
from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding

In [10]:
# Define documents
docs = [
    'Hey Suzann! Just wanted to let you know that Grandma and Grandpa are coming over for dinner tonight',
    'Dear Suzann, I hope this message finds you well. I am writing to inform you that your grandparents ...',
    'Amidst an ornate room, Amy\'s fingers gracefully danced across the intricate strings of her zither.',
    'Betty\'s peony bloomed exuberantly, rendering her garden with a symphony of hues, meticulously distri...',
    'The cornet is a brass instrument resembling a trumpet, invented by Michael in the early 19th century...',
    'Like the ebb and flow of the turbulent sea, dysfunction paints a canvas of shadows and light. While ...',
    'Teresa\'s heart raced as she lay on the operating table, a wave of trepidation washing over her.',
    'Jane and her friends set sail on their yawl, the Sea Breeze, for a day of adventure.',
    'Today, Angela and I explored the world of gastronomy at a local food festival',
    'Title: The Bronco: A Sustainable Trailblazer The Bronco, an iconic American off-road vehicle by For...'
]

In [13]:
# Integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)

# Pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[13, 7, 18, 16, 30, 34, 27, 34, 22, 39, 21, 7, 42, 7, 14, 1, 40, 10], [38, 7, 17, 32, 13, 11, 19, 27, 43, 17, 15, 12, 30, 35, 27, 22, 18, 10], [15, 43, 47, 25, 25, 8, 43, 46, 30, 3, 24, 16, 31, 40, 32], [21, 4, 15, 18, 29, 40, 22, 33, 4, 42, 31, 41, 18, 47], [3, 2, 10, 4, 15, 6, 45, 4, 40, 23, 46, 39, 41, 3, 16, 34, 12], [4, 3, 27, 21, 41, 31, 3, 44, 13, 34, 15, 4, 15, 31, 41, 21, 5, 39], [34, 44, 42, 19, 43, 4, 48, 3, 6, 1, 4, 12, 31, 30, 44, 14, 40], [47, 21, 40, 16, 34, 41, 48, 14, 7, 3, 13, 31, 1, 4, 37, 31, 29], [5, 20, 21, 17, 35, 3, 40, 31, 18, 39, 4, 34, 6, 30], [28, 3, 38, 4, 15, 26, 3, 38, 43, 27, 22, 25, 22, 23, 46, 1]]
[[14  1 40 10]
 [27 22 18 10]
 [16 31 40 32]
 [31 41 18 47]
 [ 3 16 34 12]
 [41 21  5 39]
 [30 44 14 40]
 [ 4 37 31 29]
 [ 4 34  6 30]
 [22 23 46  1]]


In [22]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from numpy import array

# Load the CSV file
csv_file = '/kaggle/input/llm-prompt-recovery-data-gemini-and-gemma/gemini_data_set_prompt_recover_3.csv'
data = pd.read_csv(csv_file)

# Extract text data from the 'data' column
train_docs = data['rewritten_text'].tolist()

# Create a tokenizer object
tokenizer = Tokenizer()
# Fit the tokenizer on the training documents
tokenizer.fit_on_texts(train_docs)
# Encode the training documents to sequences of integers
encoded_docs = tokenizer.texts_to_sequences(train_docs)

# Pad sequences to ensure uniform length
max_length = max([len(s.split()) for s in train_docs])  # Find the maximum length
Xtrain = pad_sequences(encoded_docs, maxlen=max_length, padding='post')  # Pad sequences

# Define training labels
# Assuming 'label' column contains the labels for each document
ytrain = data['original_text'].values

In [23]:
# define vocabulary size (largest integer value)
vocab_size = len(tokenizer.word_index) + 1

In [28]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Define problem
vocab_size = len(word_index) + 1
max_length = 100

# Pad sequences
padded_sequences = pad_sequences(tokenizer.texts_to_sequences(train_docs), maxlen=max_length)

# Define the model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=8))  # No need for input_length here
model.add(Flatten())
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(padded_sequences, np.array(data['target']), epochs=10, batch_size=32, validation_split=0.2)  

# Summarize the model
print(model.summary())

Epoch 1/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.0000e+00 - loss: -421.1597 - val_accuracy: 0.0028 - val_loss: -5554.8223
Epoch 2/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: -12321.8516 - val_accuracy: 0.0028 - val_loss: -52276.2930
Epoch 3/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: -73848.8281 - val_accuracy: 0.0028 - val_loss: -178374.5781
Epoch 4/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: -219712.8438 - val_accuracy: 0.0028 - val_loss: -391833.8750
Epoch 5/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: -448591.6875 - val_accuracy: 0.0028 - val_loss: -695440.5625
Epoch 6/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: -733531.8750 - val_accuracy: 0.0028 - val_loss: -1032253.0625
Epoch 7/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - loss: -1076257.8750 - val_accuracy: 0.0028 - val_loss: -1440309.0000
Epoch 8/10


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ (None, 100, 8)         │       100,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 800)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           801 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 304,373 (1.16 MB)

 Trainable params: 101,457 (396.32 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 202,916 (792.64 KB)

None


In [32]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Read the CSV file
data = pd.read_csv('/kaggle/input/llm-prompt-recovery-data-gemini-and-gemma/gemini_data_set_prompt_recover_3.csv')

# Split the data into features (X) and target (y)
X = data['rewritten_text']
y = data['original_text']  # Assuming 'labels' column contains the target labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad the tokenized sequences
max_length = 100  # Define the maximum length of sequences
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding='post')

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Encode the labels
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=8))  # Remove input_length parameter
model.add(Flatten())
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit the model
model.fit(X_train_padded, y_train_encoded, epochs=10, verbose=2)

# Evaluate
loss, acc = model.evaluate(X_test_padded, y_test_encoded, verbose=0)
print('Test Accuracy: %.2f%%' % (acc * 100))

Epoch 1/10
46/46 - 1s - 25ms/step - accuracy: 0.0028 - loss: -2.2155e+02
Epoch 2/10
46/46 - 0s - 4ms/step - accuracy: 0.0028 - loss: -3.3435e+03
Epoch 3/10
46/46 - 0s - 4ms/step - accuracy: 0.0028 - loss: -1.3082e+04
Epoch 4/10
46/46 - 0s - 4ms/step - accuracy: 0.0028 - loss: -3.4405e+04
Epoch 5/10
46/46 - 0s - 4ms/step - accuracy: 0.0028 - loss: -6.6601e+04
Epoch 6/10
46/46 - 0s - 4ms/step - accuracy: 0.0028 - loss: -1.1472e+05
Epoch 7/10
46/46 - 0s - 6ms/step - accuracy: 0.0028 - loss: -1.7453e+05
Epoch 8/10
46/46 - 0s - 4ms/step - accuracy: 0.0028 - loss: -2.4976e+05
Epoch 9/10
46/46 - 0s - 6ms/step - accuracy: 0.0028 - loss: -3.3763e+05
Epoch 10/10
46/46 - 0s - 4ms/step - accuracy: 0.0028 - loss: -4.5105e+05
Test Accuracy: 0.28%


0.28% were classified correctly by the model

I dont get label = array in this dataframe so it not completed that code snippets